In [12]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
servico = Service(ChromeDriverManager().install())

chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36")

navegador = webdriver.Chrome(service=servico,options=chrome_options)

navegador.get('https://www.casasbahia.com.br/c/eletroportateis?&filtro=categoria^d:c73&filtro=avaliacao^d:r5&utm_campaign=DescontoEspecial,DescontoEspecial,DescontoEspecial,DescontoEspecial&utm_medium=BuscaOrganica,BuscaOrganica,BuscaOrganica,BuscaOrganica&utm_source=Google,Google,Google,Google&page=1')

button = navegador.find_element(By.CLASS_NAME, 'styles__SeeMoreButton-sc-d3ac2015-6')
button.click()
brand_section = navegador.find_element(By.CSS_SELECTOR, "[data-testid='brand-filters']")

brands = brand_section.find_elements(By.CLASS_NAME, 'styles__LabelText-sc-d3ac2015-4')
marcas = []
for brand in brands:
    marcas.append(brand.text)

print(marcas)

navegador.quit()

cols = ['Nome_produto','avaliacao','qtd_avaliacoes','possui_desconto','desconto','preco','marca','tipo']
data = []

for i in range(1,70):
    if i==20:continue
    chrome_options = Options()
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36")

    navegador = webdriver.Chrome(service=servico,options=chrome_options)

    print(f'Pegando dados da pagina {i}...')
    navegador.get(f'https://www.casasbahia.com.br/c/eletroportateis?&filtro=categoria^d:c73&filtro=avaliacao^d:r5&utm_campaign=DescontoEspecial,DescontoEspecial,DescontoEspecial,DescontoEspecial&utm_medium=BuscaOrganica,BuscaOrganica,BuscaOrganica,BuscaOrganica&utm_source=Google,Google,Google,Google&page={i}')
    time.sleep(5)
    cards = navegador.find_elements(By.CLASS_NAME, 'css-gd7wmb')

    for card in cards:
        card_data=[]
        titulo = card.find_element(By.TAG_NAME,'h3').text
        card_data.append(titulo)
        avaliacao = float(card.find_element(By.CLASS_NAME,'css-1vmkvrm').text.split()[-4])
        card_data.append(avaliacao)
        qtd_avaliacoes = float(card.find_element(By.CLASS_NAME,'product-card__reviews-count-text').text[1:-1])
        card_data.append(qtd_avaliacoes)
        try:
            discount = card.find_element(By.CLASS_NAME, 'product-card__discount-wrapper').text.split()
            
            if len(discount) == 0:
                card_data.append(False)
                discount_prc = "0%"  
                card_data.append(discount_prc)
            else:
                card_data.append(True)
                discount_prc = discount[2]  
                card_data.append(discount_prc)
        except:
            card_data.append(False)
            discount_prc = "0%"  
            card_data.append(discount_prc)
        price = card.find_element(By.CLASS_NAME,'product-card__highlight-price').text.split()[1].replace(',','.')
        card_data.append(price)

        confirm_brand=False
        for brand in marcas:
            if brand in titulo or brand.lower() in titulo: 
                card_data.append(brand)
                confirm_brand=True
                break
        if confirm_brand==False:
            card_data.append("Outras")
            print(card_data)
        data.append(card_data)
        tipo = titulo.split()[0]
        card_data.append(tipo)
    navegador.quit() 
print(data)
df = pd.DataFrame(data,columns=cols)
if 'id_produto' not in df.columns:
    df = df.assign(id_produto=range(1, len(df) + 1))

# Reordenar as colunas para garantir que 'id_produto' seja a primeira coluna
df = df[['id_produto'] + [col for col in df.columns if col != 'id_produto']]
display(df)
df.to_csv('casas_bahia_eletrodomesticos.csv', index=False)


['Electrolux', 'Mondial', 'Britânia', 'Oster', 'Philco', 'Cadence', 'Black+Decker', 'Philips Walita', 'EOS', 'Arno', 'Wap', 'Agratto', 'Multilaser', 'IBBL', 'Consul', 'Lenoxx', 'Mallory', 'Black', 'Esmaltec', 'Mistral SUP', 'Elgin', 'TOYS', 'Philips', 'TRES', 'Hidro Filtros', 'Fischer', 'Midea', 'Latina Eletrodomésticos', 'Lorenzetti', 'Singer', 'MICROMAX', 'LAYR', 'Suggar', 'Amvox', 'Colormaq', 'Nespresso', 'Mueller', 'Hq', 'Europa', 'Tramontina', 'Nardelli', 'Gaabor', 'Karina Bebedouros', 'Fama', 'WFS', 'Hamilton Beach', 'Acquamar', '3 Corações', '3 M', 'Dako', 'Nescafé Dolce Gusto', 'Saro', 'IMPORT WAY', 'MAXIS', 'All Clean', 'Kärcher', 'Janome', 'Vonder', 'Yamata', 'Acqua bios', 'Mebrasi', 'Ventisol', 'Frisbel', 'KitchenAid', 'In Brasil', 'Bracob', 'Diluka Power', 'Continental', 'Xiaomi', 'Top Life', '24 HORAS FILTROS', 'Fit Mobel', 'Titã', 'Dolce Gusto', 'Gmold', 'Cerâmica Stefani', 'Itatiaia', 'Atlas', 'Samsung', 'Izumi', 'Skymsen', 'Brother', 'Ariete', 'Sun Special', 'Karina', '

,id_produto,Nome_produto,avaliacao,qtd_avaliacoes,possui_desconto,desconto,preco,marca,tipo
0,1,Fritadeira Elétrica Sem Óleo Air Fryer Mondial...,5.0,9303.0,True,20%,265.90,Mondial,Fritadeira
1,2,Liquidificador Philco PH900 Preto 1200W com 12...,5.0,16878.0,True,25%,129.00,Philco,Liquidificador
2,3,Sanduicheira Fast Grill e Sandwich Mondial S-1...,5.0,3468.0,True,28%,69.90,Mondial,Sanduicheira
3,4,Liquidificador Mondial Turbo Power L-99 FB Pre...,5.0,8588.0,True,7%,105.90,Mondial,Liquidificador
4,5,Fritadeira Elétrica Sem Óleo Air Fryer Mondial...,5.0,882.0,True,20%,399.00,Mondial,Fritadeira
...,...,...,...,...,...,...,...,...,...
1355,1356,Aspirador de Pó Robô Wap Robot W100 com Mop Au...,4.5,551.0,True,7%,549.99,Wap,Aspirador
1356,1357,Purificador de Água Electrolux PE11B com Paine...,4.5,2479.0,False,0%,649.00,Electrolux,Purificador
1357,1358,Vaporizador de Roupas Electrolux Expert EGS20 ...,4.5,99.0,True,21%,619.90,Electrolux,Vaporizador
1358,1359,Purificador de Água Colormaq Eletrônico CPUELS...,4.5,523.0,False,0%,447.10,Colormaq,Purificador


In [13]:
display(df.head(50))

,id_produto,Nome_produto,avaliacao,qtd_avaliacoes,possui_desconto,desconto,preco,marca,tipo
0,1,Fritadeira Elétrica Sem Óleo Air Fryer Mondial...,5.0,9303.0,True,20%,265.90,Mondial,Fritadeira
1,2,Liquidificador Philco PH900 Preto 1200W com 12...,5.0,16878.0,True,25%,129.00,Philco,Liquidificador
2,3,Sanduicheira Fast Grill e Sandwich Mondial S-1...,5.0,3468.0,True,28%,69.90,Mondial,Sanduicheira
3,4,Liquidificador Mondial Turbo Power L-99 FB Pre...,5.0,8588.0,True,7%,105.90,Mondial,Liquidificador
4,5,Fritadeira Elétrica Sem Óleo Air Fryer Mondial...,5.0,882.0,True,20%,399.00,Mondial,Fritadeira
5,6,Liquidificador Mondial Power- L-550 com 2 Velo...,5.0,8709.0,True,22%,62.91,Mondial,Liquidificador
6,7,Liquidificador Oster OLIQ610 15 Velocidades 14...,5.0,8801.0,True,21%,169.90,Oster,Liquidificador
7,8,Fritadeira Elétrica Sem Óleo Air Fryer Mondial...,5.0,11987.0,True,23%,239.90,Mondial,Fritadeira
8,9,Fritadeira Elétrica Sem Óleo Air Fryer Oven 2 ...,5.0,734.0,True,30%,599.00,Mondial,Fritadeira
9,10,Aspirador de Pó Vertical 2 em 1 Mondial Turbo ...,5.0,14173.0,True,16%,141.01,Mondial,Aspirador
